In [ ]:
import json
import boto3
import csv
from io import StringIO

def transform_spotify_data(data):
    # Extract track details
    tracks = data.get('tracks', [])
    transformed_data = []
    
    for track in tracks:
        track_info = {
            "Track Name": track.get("name", ""),
            "Album Name": track.get("album", {}).get("name", ""),
            "Artist Name": ", ".join([artist.get("name", "") for artist in track.get("artists", [])]),
            "Duration (ms)": track.get("duration_ms", 0),
            "Popularity": track.get("popularity", 0),
            "Spotify URL": track.get("external_urls", {}).get("spotify", "")
        }
        transformed_data.append(track_info)
    
    return transformed_data

def write_to_csv(data):
    # Write transformed data to a CSV format in memory
    csv_buffer = StringIO()
    fieldnames = ["Track Name", "Album Name", "Artist Name", "Duration (ms)", "Popularity", "Spotify URL"]
    writer = csv.DictWriter(csv_buffer, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerows(data)
    return csv_buffer.getvalue()

def lambda_handler(event, context):
    s3 = boto3.client('s3')
    
    # Source and target bucket details
    source_bucket = "spotify-extractdata"  # Replace with your source bucket name
    target_bucket = "spotify-transformdata"  # Replace with your target bucket name
    
    # Get the file details from the event
    for record in event['Records']:
        source_key = record['s3']['object']['key']
        
        # Read the JSON file from the source bucket
        response = s3.get_object(Bucket=source_bucket, Key=source_key)
        raw_data = json.loads(response['Body'].read())
        
        # Transform the data
        transformed_data = transform_spotify_data(raw_data)
        
        # Convert the transformed data to CSV
        csv_content = write_to_csv(transformed_data)
        
        # Save the transformed CSV file to the target bucket
        target_key = f"transformed_data/{source_key.split('/')[-1].replace('.json', '.csv')}"
        s3.put_object(Bucket=target_bucket, Key=target_key, Body=csv_content)
    
    return {
        'statusCode': 200,
        'body': f"File transformed and saved to {target_bucket}/{target_key}"
    }
